# Задания с решениями по БД Startup Investments (Kaggle)

Ссылка на датасет - https://www.kaggle.com/justinas/startup-investments

1. Отобразите все записи из таблицы company по компаниям, которые закрылись.

In [ ]:
SELECT *
FROM company
WHERE company.status = 'closed'

2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.

In [ ]:
SELECT c.funding_total
FROM company AS c
WHERE c.category_code = 'news'
    AND c.country_code = 'USA'
ORDER BY c.funding_total DESC

3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:

SELECT SUM(price_amount)
FROM acquisition AS a
WHERE a.term_code = 'cash'
    AND EXTRACT(YEAR FROM CAST(acquired_at AS timestamp)) BETWEEN 2011 AND 2013

4. Отобразите имя, фамилию и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:

SELECT first_name,
       last_name,
       network_username
FROM people
WHERE network_username LIKE 'Silver%'


5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
FROM people
WHERE network_username LIKE '%money%'
    AND last_name LIKE 'K%'


6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:

SELECT country_code,
       SUM(funding_total)
FROM company AS c
GROUP BY country_code
ORDER BY SUM(funding_total) DESC


7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) != 0
  AND MIN(raised_amount) != MAX(raised_amount)

8. Создайте поле с категориями:
•	Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
•	Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
•	Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
       CASE
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies >= 20 AND invested_companies < 100
           THEN 'middle_activity'
           WHEN invested_companies < 20 THEN 'low_activity'
       END AS category
FROM fund


9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds)) AS avg_rounds
FROM fund
GROUP BY activity
ORDER BY avg_rounds

10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) !=0
ORDER BY AVG(invested_companies) DESC, country_code
LIMIT 10

11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT first_name,
       last_name,
       instituition
FROM people AS p LEFT OUTER JOIN education As e ON p.id = e.person_id

12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT c.name,
       COUNT(DISTINCT e.instituition)
FROM company AS c LEFT OUTER JOIN people AS p ON c.id = p.company_id
                       INNER JOIN education AS e ON p.id = e.person_id
GROUP BY c.name
ORDER BY COUNT(DISTINCT e.instituition) DESC
LIMIT 5

13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT c.name
FROM company AS c INNER JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.status = 'closed'
    AND fr.is_first_round = 1
    AND fr.is_last_round = 1

14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT DISTINCT p.id
FROM people AS p INNER JOIN company AS c ON p.company_id = c.id
WHERE c.name in (
                SELECT DISTINCT c.name
                FROM company AS c INNER JOIN funding_round AS fr ON c.id = fr.company_id
                WHERE c.status = 'closed'
                    AND fr.is_first_round = 1
                    AND fr.is_last_round = 1)

15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT person_id,
    		   instituition
FROM education
WHERE person_id IN (SELECT DISTINCT p.id
                                        FROM people AS p INNER JOIN company AS c ON p.company_id = c.id
                                       WHERE c.name in (
                                SELECT DISTINCT c.name
                   FROM company AS c INNER JOIN funding_round AS fr ON c.id =    
                   fr.company_id
                                WHERE c.status = 'closed'
                                    AND fr.is_first_round = 1
                                    AND fr.is_last_round = 1))

16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды. (Условие из второго предложения не проверяется!)

In [ ]:
SELECT DISTINCT person_id,
             COUNT(instituition)
FROM education
WHERE person_id IN (SELECT DISTINCT p.id
                    FROM people AS p INNER JOIN company AS c ON p.company_id = c.id
                    WHERE c.name in (
                                SELECT DISTINCT c.name
                                FROM company AS c INNER JOIN funding_round AS fr ON c.id = fr.company_id
                                WHERE c.status = 'closed'
                                    AND fr.is_first_round = 1
                                    AND fr.is_last_round = 1))
GROUP BY person_id

17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
WITH
a AS (SELECT DISTINCT person_id,
             COUNT(instituition) AS inst_count
FROM education
WHERE person_id IN (SELECT DISTINCT p.id
                    FROM people AS p INNER JOIN company AS c ON p.company_id = c.id
                    WHERE c.name in (
                                SELECT DISTINCT c.name
                                FROM company AS c INNER JOIN funding_round AS fr ON c.id = fr.company_id
                                WHERE c.status = 'closed'
                                    AND fr.is_first_round = 1
                                    AND fr.is_last_round = 1))
GROUP BY person_id)

SELECT SUM(a.inst_count) / COUNT(a.person_id)
FROM a

18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.

In [ ]:
WITH a AS (SELECT  person_id,
             COUNT(instituition) AS inst_count
FROM education
GROUP BY person_id),

b AS (SELECT p.id AS pid
      FROM people AS p INNER JOIN company AS c ON p.company_id = c.id
      WHERE c.name = 'Socialnet')

SELECT SUM(a.inst_count) / COUNT(a.person_id)
FROM a INNER JOIN b ON a.person_id = b.pid

19. Составьте таблицу из полей:
•	name_of_fund — название фонда;
•	name_of_company — название компании;
•	amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM fund AS f INNER JOIN investment As i ON f.id = i.fund_id
               INNER JOIN funding_round As fr ON i.funding_round_id = fr.id
               INNER JOIN company AS c ON fr.company_id = c.id
WHERE c.milestones > 6
    AND EXTRACT(YEAR FROM CAST(fr.funded_at AS timestamp)) BETWEEN 2012 AND 2013

20. Выгрузите таблицу, в которой будут такие поля:
•	название компании-покупателя;
•	сумма сделки;
•	название компании, которую купили;
•	сумма инвестиций, вложенных в купленную компанию;
•	доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT c_ing.name AS acquiring_company,
       a.price_amount,
       c_ed.name AS acquired_company,
       c_ed.funding_total,
       ROUND(a.price_amount / c_ed.funding_total)
FROM acquisition AS a INNER JOIN company AS c_ing ON a.acquiring_company_id = c_ing.id
                      INNER JOIN company AS c_ed ON a.acquired_company_id = c_ed.id
WHERE a.price_amount != 0
    AND c_ed.funding_total !=0
ORDER BY a.price_amount DESC, acquired_company
LIMIT 10

21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name,
       EXTRACT(MONTH FROM CAST(fr.funded_at AS timestamp)) AS funding_month
FROM company AS c INNER JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.category_code = 'social'
    AND EXTRACT(YEAR FROM CAST(fr.funded_at AS timestamp)) BETWEEN 2010 AND 2013
    AND fr.raised_amount !=0

22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
•	номер месяца, в котором проходили раунды;
•	количество уникальных названий фондов из США, которые инвестировали в этом месяце;
•	количество компаний, купленных за этот месяц;
•	общая сумма сделок по покупкам в этом месяце.

In [ ]:

SELECT x.*,
       y.acquired_companies_count,
       y.sum_amount
FROM
  (SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS timestamp)) AS funding_month,
       COUNT(DISTINCT f.name) AS funds_count
   FROM funding_round AS fr INNER JOIN investment AS i ON fr.id = i.funding_round_id
                            INNER JOIN fund AS f ON i.fund_id = f.id
   WHERE f.country_code = 'USA'
       AND EXTRACT(YEAR FROM CAST(fr.funded_at AS timestamp)) BETWEEN 2010 AND 2013
   GROUP BY funding_month) AS x
LEFT OUTER JOIN
  (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS timestamp)) AS acquired_month,
          COUNT(acquired_company_id) AS acquired_companies_count,
          SUM(price_amount) AS sum_amount
   FROM acquisition
   WHERE EXTRACT(YEAR FROM CAST(acquired_at AS timestamp)) BETWEEN 2010 AND 2013
   GROUP BY acquired_month
   ORDER BY acquired_month) AS y 
   ON x.funding_month = y.acquired_month

23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
     inv_2011 AS (SELECT country_code,
                         SUM(funding_total) / COUNT(funding_total) AS sum2011
                  FROM company AS c 
                  WHERE  EXTRACT(YEAR FROM CAST(c.founded_at AS timestamp)) = 2011
                  GROUP BY country_code), 
     inv_2012 AS (SELECT country_code,
                         SUM(funding_total) / COUNT(funding_total) AS sum2012
                  FROM company AS c 
                  WHERE  EXTRACT(YEAR FROM CAST(c.founded_at AS timestamp)) = 2012
                  GROUP BY country_code),
     inv_2013 AS (SELECT country_code,
                         SUM(funding_total) / COUNT(funding_total) AS sum2013
                  FROM company AS c 
                  WHERE  EXTRACT(YEAR FROM CAST(c.founded_at AS timestamp)) = 2013
                  GROUP BY country_code) 
     
     
SELECT inv_2011.country_code,
       inv_2011.sum2011,
       inv_2012.sum2012,
       inv_2013.sum2013
FROM inv_2011
INNER JOIN inv_2012
ON inv_2011.country_code = inv_2012.country_code
INNER JOIN inv_2013
ON inv_2011.country_code = inv_2013.country_code
ORDER BY inv_2011.sum2011 DESC